In [ ]:
import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings

In [ ]:
import pandas as pd
import numpy as np
import sys

In [ ]:
design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'

In [ ]:
gui = MetalGUI(design)

In [ ]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket

design.delete_all_components()

q1 = TransmonPocket(
    design,
    'Q1',
    options=dict(pad_width='425 um',
                 pocket_height='650um'))

gui.rebuild()

In [ ]:
print(q1.qgeometry_table('poly'))

In [ ]:
design._qgeometry._tables

In [ ]:
design._qgeometry.get_component('Q1','poly')

In [ ]:
x = q1.qgeometry_table('poly')

In [ ]:
for index, row in x.iterrows():
    print(row)

In [ ]:
row = next(x.iterrows())[1]

In [ ]:
row

In [ ]:
# Extract coordinates of all polygons into a list
q1_poly=q1.qgeometry_table('poly')
coords = []
for item in q1_poly['geometry']:
    coords_new = list(item.exterior.coords)
    coords.append(coords_new)

In [ ]:
names=q1_poly['name']
print(names)

In [ ]:
# Remove last tuple from each list
rect = [] 
for item in coords:
    rect_new = item[0:-1]
    rect.append(rect_new)

In [ ]:
# Add zero to each tuple to make it compatible with the EMPro format
rect_mod = []
for i in rect:
    for j in i:
        rect_mod_new = j + (0.0,)
        rect_mod.append(rect_mod_new)

In [ ]:
print(rect_mod[0:4])

In [ ]:
# Get the original number of elements in the geometry then split the list 
x = len(q1_poly['geometry'])
j = 0
vertices = [0]*4
for i in range(x):
    vertices[i] = rect_mod[j:j+4]
    j += 4

In [ ]:
filename = "draw.py"

In [ ]:
with open(filename, 'a') as out:
    out.write(
'#Clear project space and start fresh'  + '\n'
'import empro' + '\n'
'empro.activeProject.clear()' + '\n'
'from empro.toolkit.via_designer.geometry import makePolygon, makePolyLine' + '\n'+ '\n'
        
        
'width = 0.20' + '\n'
'height = 0.10' + '\n')

In [ ]:
original_stdout = sys.stdout # Save a reference to the original standard output
with open(filename, 'a') as out:
    sys.stdout = out # Change the standard output to the file we created.
    for i in range(x):
        print(f'vertices_{i} = {vertices[i]}') 
    sys.stdout = original_stdout # Reset the standard output to its original value

In [ ]:
with open(filename, 'a') as out:
    out.write(     
    '\n' + '\n')

In [ ]:
with open(filename, 'a') as out:
    out.write(     
    'empro.activeProject.geometry.append(' + '\n')
with open(filename, 'a') as out:
    sys.stdout = out # Change the standard output to the file we created.
    for i in range(x):
        print(f'makePolygon(vertices_{i} , name="{names[i]}")')
    sys.stdout = original_stdout # Reset the standard output to its original value

In [ ]:
with open(filename, 'a') as out:
    out.write(     
    ')' + '\n' + '\n')